In [0]:
%pip install langchain>=0.1.0 langchain-groq>=0.1.0 langchain-community>=0.0.10 pandas requests python-dotenv

In [0]:
dbutils.library.restartPython()

In [0]:
import json
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional
import requests
import os

# =================== DATA LOADING ===================

class DataLoader:
    def __init__(self):
        # Define file paths
        flights_path = '/Workspace/Users/rsangramofficial@gmail.com/EDA/Travel Agaent/flights.json'
        hotels_path = '/Workspace/Users/rsangramofficial@gmail.com/EDA/Travel Agaent/hotels.json'
        places_path = '/Workspace/Users/rsangramofficial@gmail.com/EDA/Travel Agaent/places.json'
        
        self.flights = self.load_json(flights_path)
        self.hotels = self.load_json(hotels_path)
        self.places = self.load_json(places_path)
        
        # Convert to DataFrames
        self.flights_df = pd.DataFrame(self.flights)
        self.hotels_df = pd.DataFrame(self.hotels)
        self.places_df = pd.DataFrame(self.places)
        
        # Display sample data
        print("=" * 60)
        print("DATA LOADED SUCCESSFULLY")
        print("=" * 60)
        print(f"Flights: {len(self.flights_df)} records")
        print(f"Hotels: {len(self.hotels_df)} records")
        print(f"Places: {len(self.places_df)} records")
        print("\nSample Flights (first 3):")
        print(self.flights_df[['flight_id', 'airline', 'from', 'to', 'price']].head(3).to_string())
        print("\nSample Hotels (first 3):")
        print(self.hotels_df[['hotel_id', 'name', 'city', 'stars', 'price_per_night']].head(3).to_string())
        print("\nSample Places (first 3):")
        print(self.places_df[['place_id', 'name', 'city', 'type', 'rating']].head(3).to_string())
        
        # Show available cities
        print("\n" + "="*60)
        print("AVAILABLE CITIES")
        print("="*60)
        print(f"Flight Departure Cities: {sorted(self.flights_df['from'].unique().tolist())}")
        print(f"Flight Arrival Cities: {sorted(self.flights_df['to'].unique().tolist())}")
        print(f"Hotel Cities: {sorted(self.hotels_df['city'].unique().tolist())}")
        print(f"Place Cities: {sorted(self.places_df['city'].unique().tolist())}")
    
    def load_json(self, filepath):
        """Load JSON data from file"""
        try:
            with open(filepath, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            print(f"Error: File not found at {filepath}")
            return []
        except json.JSONDecodeError:
            print(f"Error: Invalid JSON in {filepath}")
            return []
    
    def get_available_routes(self):
        """Get available flight routes"""
        routes = self.flights_df[['from', 'to']].drop_duplicates()
        return routes.to_dict('records')

# =================== SIMPLE TOOLS ===================

class SimpleFlightSearchTool:
    def __init__(self, data_loader):
        self.data_loader = data_loader
    
    def search(self, from_city, to_city, max_price=None, airline=None):
        try:
            from_city = from_city.strip().title()
            to_city = to_city.strip().title()
            
            print(f"DEBUG: Searching flights from '{from_city}' to '{to_city}'")
            
            if not from_city or not to_city:
                return json.dumps({"error": "Please provide both 'from_city' and 'to_city'"})
            
            # Access flights_df through data_loader
            flights_df = self.data_loader.flights_df
            
            # Check if cities exist in data
            available_from = flights_df['from'].unique().tolist()
            available_to = flights_df['to'].unique().tolist()
            
            print(f"DEBUG: Available departure cities: {available_from}")
            print(f"DEBUG: Available arrival cities: {available_to}")
            
            # Filter flights - case insensitive
            filtered = flights_df[
                (flights_df['from'].str.strip().str.title() == from_city) &
                (flights_df['to'].str.strip().str.title() == to_city)
            ].copy()
            
            print(f"DEBUG: Found {len(filtered)} flights")
            
            if filtered.empty:
                # Try to find similar cities
                similar_from = [city for city in available_from if from_city.lower() in city.lower()]
                similar_to = [city for city in available_to if to_city.lower() in city.lower()]
                
                available_routes = self.data_loader.get_available_routes()
                
                return json.dumps({
                    "error": f"No flights found from {from_city} to {to_city}",
                    "similar_from": similar_from[:3],
                    "similar_to": similar_to[:3],
                    "available_routes": available_routes[:5]
                })
            
            # Apply filters
            if max_price:
                filtered = filtered[filtered['price'] <= float(max_price)]
            
            if airline:
                filtered = filtered[filtered['airline'].str.contains(airline, case=False, na=False)]
            
            # Sort by price (cheapest first)
            filtered = filtered.sort_values('price')
            
            # Format results
            results = []
            for _, flight in filtered.head(5).iterrows():
                duration = self.calculate_duration(flight.get('departure_time', ''), flight.get('arrival_time', ''))
                
                results.append({
                    'flight_id': flight.get('flight_id', 'N/A'),
                    'airline': flight.get('airline', 'N/A'),
                    'route': f"{flight.get('from', 'N/A')} → {flight.get('to', 'N/A')}",
                    'departure': flight.get('departure_time', 'N/A')[:16].replace('T', ' ') if flight.get('departure_time') else 'N/A',
                    'arrival': flight.get('arrival_time', 'N/A')[:16].replace('T', ' ') if flight.get('arrival_time') else 'N/A',
                    'price': f"₹{flight.get('price', 0)}",
                    'duration': duration
                })
            
            response = {
                'total_options': len(filtered),
                'cheapest': {
                    'airline': filtered.iloc[0].get('airline', 'N/A'),
                    'price': f"₹{filtered.iloc[0].get('price', 0)}",
                    'departure': filtered.iloc[0].get('departure_time', 'N/A')[:16].replace('T', ' ') if filtered.iloc[0].get('departure_time') else 'N/A'
                },
                'top_flights': results[:3]
            }
            
            return json.dumps(response, indent=2)
            
        except Exception as e:
            return json.dumps({"error": f"Error searching flights: {str(e)}"})
    
    def calculate_duration(self, dep_time_str, arr_time_str):
        try:
            if not dep_time_str or not arr_time_str:
                return "N/A"
            
            # Parse datetime strings
            dep = datetime.fromisoformat(dep_time_str.replace('Z', '+00:00'))
            arr = datetime.fromisoformat(arr_time_str.replace('Z', '+00:00'))
            
            duration = arr - dep
            total_hours = duration.total_seconds() / 3600
            
            hours = int(total_hours)
            minutes = int((total_hours - hours) * 60)
            
            return f"{hours}h {minutes}m"
        except:
            return "N/A"

class SimpleHotelSearchTool:
    def __init__(self, data_loader):
        self.data_loader = data_loader
    
    def search(self, city, min_stars=None, max_price=None, amenities=None):
        try:
            city = city.strip().title()
            
            if not city:
                return json.dumps({"error": "Please provide 'city' parameter"})
            
            hotels_df = self.data_loader.hotels_df
            
            # Filter hotels by city
            filtered = hotels_df[
                hotels_df['city'].str.strip().str.title() == city
            ].copy()
            
            if filtered.empty:
                available_cities = hotels_df['city'].unique().tolist()
                return json.dumps({
                    "error": f"No hotels found in {city}",
                    "available_cities": available_cities[:5]
                })
            
            # Apply filters
            if min_stars:
                filtered = filtered[filtered['stars'] >= int(min_stars)]
            
            if max_price:
                filtered = filtered[filtered['price_per_night'] <= float(max_price)]
            
            if amenities:
                amenities_list = [a.lower().strip() for a in amenities]
                def has_amenities(hotel_amenities):
                    if not isinstance(hotel_amenities, list):
                        return False
                    hotel_amenities_lower = [str(a).lower().strip() for a in hotel_amenities]
                    return any(amenity in hotel_amenities_lower for amenity in amenities_list)
                
                filtered = filtered[filtered['amenities'].apply(has_amenities)]
            
            filtered = filtered.sort_values(['stars', 'price_per_night'], ascending=[False, True])
            
            results = []
            for _, hotel in filtered.head(5).iterrows():
                value_score = (hotel.get('stars', 0) * 20) / max(hotel.get('price_per_night', 1000) / 1000, 1)
                
                results.append({
                    'hotel_id': hotel.get('hotel_id', 'N/A'),
                    'name': hotel.get('name', 'N/A'),
                    'stars': hotel.get('stars', 0),
                    'star_rating': '⭐' * hotel.get('stars', 0),
                    'price_per_night': f"₹{hotel.get('price_per_night', 0)}",
                    'amenities': ', '.join(hotel.get('amenities', [])[:3]),
                    'value_score': round(value_score, 2)
                })
            
            response = {
                'total_hotels': len(filtered),
                'city': city,
                'recommendations': results[:3]
            }
            
            return json.dumps(response, indent=2)
            
        except Exception as e:
            return json.dumps({"error": f"Error searching hotels: {str(e)}"})

class SimplePlacesSearchTool:
    def __init__(self, data_loader):
        self.data_loader = data_loader
    
    def search(self, city, place_type=None, min_rating=None):
        try:
            city = city.strip().title()
            
            if not city:
                return json.dumps({"error": "Please provide 'city' parameter"})
            
            places_df = self.data_loader.places_df
            
            filtered = places_df[
                places_df['city'].str.strip().str.title() == city
            ].copy()
            
            if filtered.empty:
                available_cities = places_df['city'].unique().tolist()
                return json.dumps({
                    "error": f"No attractions found in {city}",
                    "available_cities": available_cities[:5]
                })
            
            if place_type:
                filtered = filtered[filtered['type'].str.lower() == place_type.lower()]
            
            if min_rating:
                filtered = filtered[filtered['rating'] >= float(min_rating)]
            
            filtered = filtered.sort_values('rating', ascending=False)
            
            results = []
            for _, place in filtered.head(8).iterrows():
                results.append({
                    'name': place.get('name', 'N/A'),
                    'type': place.get('type', 'N/A'),
                    'rating': place.get('rating', 0),
                    'stars': '★' * int(place.get('rating', 0))
                })
            
            response = {
                'city': city,
                'total_places': len(filtered),
                'top_rated': results[:5]
            }
            
            return json.dumps(response, indent=2)
            
        except Exception as e:
            return json.dumps({"error": f"Error searching places: {str(e)}"})

class SimpleWeatherTool:
    def __init__(self):
        pass
    
    def get_weather(self, city, date=None):
        try:
            if not date:
                date = datetime.now().strftime('%Y-%m-%d')
            
            city = city.strip().title()
            
            if not city:
                return json.dumps({"error": "Please provide 'city' parameter"})
            
            city_coords = {
                'Delhi': (28.7041, 77.1025),
                'Mumbai': (19.0760, 72.8777),
                'Goa': (15.2993, 74.1240),
                'Bangalore': (12.9716, 77.5946),
                'Chennai': (13.0827, 80.2707),
                'Hyderabad': (17.3850, 78.4867),
                'Kolkata': (22.5726, 88.3639),
                'Jaipur': (26.9124, 75.7873)
            }
            
            if city not in city_coords:
                available_cities = list(city_coords.keys())
                return json.dumps({
                    "error": f"Weather data not available for {city}",
                    "available_cities": available_cities
                })
            
            lat, lon = city_coords[city]
            
            url = "https://api.open-meteo.com/v1/forecast"
            params_api = {
                'latitude': lat,
                'longitude': lon,
                'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum,weathercode',
                'timezone': 'auto',
                'forecast_days': 1
            }
            
            response = requests.get(url, params=params_api, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            if 'daily' in data:
                temp_max = data['daily']['temperature_2m_max'][0]
                temp_min = data['daily']['temperature_2m_min'][0]
                precipitation = data['daily']['precipitation_sum'][0]
                weather_code = data['daily']['weathercode'][0]
                
                weather_desc = self.get_weather_description(weather_code)
                recommendation = self.get_weather_recommendation(weather_code)
                
                response_data = {
                    'city': city,
                    'date': date,
                    'temperature': f"{temp_min}°C - {temp_max}°C",
                    'weather': weather_desc,
                    'precipitation': f"{precipitation} mm",
                    'recommendation': recommendation
                }
                
                return json.dumps(response_data, indent=2)
            else:
                return json.dumps({"error": f"Weather forecast not available for {city}"})
                
        except requests.exceptions.RequestException as e:
            return json.dumps({"error": f"Error fetching weather data: {str(e)}"})
        except Exception as e:
            return json.dumps({"error": f"Error: {str(e)}"})
    
    def get_weather_description(self, code):
        weather_codes = {
            0: "Clear sky ☀️",
            1: "Mainly clear 🌤️",
            2: "Partly cloudy ⛅",
            3: "Overcast ☁️",
            45: "Foggy 🌫️",
            48: "Foggy 🌫️",
            51: "Light drizzle 🌧️",
            53: "Moderate drizzle 🌧️",
            55: "Dense drizzle 🌧️",
            61: "Slight rain 🌦️",
            63: "Moderate rain 🌧️",
            65: "Heavy rain 🌧️",
            71: "Slight snow ❄️",
            73: "Moderate snow ❄️",
            75: "Heavy snow ❄️",
            80: "Rain showers 🌦️",
            81: "Heavy rain showers 🌧️",
            82: "Violent rain showers ⛈️",
            95: "Thunderstorm ⛈️"
        }
        return weather_codes.get(code, "Unknown")
    
    def get_weather_recommendation(self, code):
        if code in [0, 1, 2]:
            return "Perfect for outdoor activities! Apply sunscreen."
        elif code == 3:
            return "Good for sightseeing. Light jacket recommended."
        elif 45 <= code <= 55:
            return "Carry an umbrella. Good for indoor attractions."
        elif 61 <= code <= 65:
            return "Rain expected. Plan indoor activities."
        elif code >= 80:
            return "Rain showers expected. Waterproof gear recommended."
        elif code >= 95:
            return "Thunderstorm warning. Stay indoors."
        return "Check weather updates."

class SimpleBudgetCalculatorTool:
    def __init__(self):
        pass
    
    def calculate(self, flight_price=0, hotel_price_per_night=2000, num_nights=3, 
                  food_per_day=1000, transport_per_day=500, activities_per_day=1000, 
                  miscellaneous=1000):
        try:
            hotel_total = hotel_price_per_night * num_nights
            food_total = food_per_day * num_nights
            transport_total = transport_per_day * num_nights
            activities_total = activities_per_day * num_nights
            
            subtotal = hotel_total + food_total + transport_total + activities_total + miscellaneous
            total = flight_price + subtotal
            
            per_day_cost = subtotal / num_nights if num_nights > 0 else 0
            
            response = {
                'budget_breakdown': {
                    'Flight': f"₹{flight_price:,.0f}",
                    'Accommodation': f"₹{hotel_total:,.0f}",
                    'Food & Dining': f"₹{food_total:,.0f}",
                    'Local Transport': f"₹{transport_total:,.0f}",
                    'Activities': f"₹{activities_total:,.0f}",
                    'Miscellaneous': f"₹{miscellaneous:,.0f}"
                },
                'total': f"₹{total:,.0f}",
                'per_day_average': f"₹{per_day_cost:,.0f}",
                'money_saving_tips': [
                    "Book in advance for better prices",
                    "Travel during weekdays",
                    "Use public transport",
                    "Look for combo tickets",
                    "Try local street food for authentic experience"
                ]
            }
            
            return json.dumps(response, indent=2)
            
        except Exception as e:
            return json.dumps({"error": f"Error calculating budget: {str(e)}"})

# =================== IMPROVED INTERFACE ===================

def create_simple_interface():
    """Create a simple text-based interface for Databricks"""
    
    print("\n" + "="*70)
    print("🤖 TRAVEL PLANNING ASSISTANT")
    print("="*70)
    
    try:
        print("\nLoading data...")
        data_loader = DataLoader()
        
        print("\nCreating tools...")
        flight_tool = SimpleFlightSearchTool(data_loader)
        hotel_tool = SimpleHotelSearchTool(data_loader)
        places_tool = SimplePlacesSearchTool(data_loader)
        weather_tool = SimpleWeatherTool()
        budget_tool = SimpleBudgetCalculatorTool()
        
        print("✅ Tools created successfully!")
        
        while True:
            print("\n" + "-"*50)
            print("MAIN MENU")
            print("-"*50)
            print("1. Search flights")
            print("2. Search hotels")
            print("3. Search tourist attractions")
            print("4. Check weather forecast")
            print("5. Calculate trip budget")
            print("6. Create complete travel plan")
            print("7. Show available cities")
            print("8. Exit")
            print("-"*50)
            
            choice = input("\n👉 Select option (1-8): ").strip()
            
            if choice == '8':
                print("\n👋 Thank you for using Travel Planning Assistant!")
                print("Safe travels! ✈️")
                break
            
            try:
                if choice == '1':
                    print("\n" + "="*50)
                    print("✈️ FLIGHT SEARCH")
                    print("="*50)
                    from_city = input("Departure city: ").strip()
                    to_city = input("Destination city: ").strip()
                    
                    if not from_city or not to_city:
                        print("❌ Please provide both cities.")
                        continue
                    
                    max_price = input("Maximum price (₹, optional): ").strip()
                    airline = input("Preferred airline (optional): ").strip()
                    
                    result = flight_tool.search(from_city, to_city, 
                                               max_price=float(max_price) if max_price else None,
                                               airline=airline if airline else None)
                    
                    print("\n" + "="*50)
                    print("RESULTS:")
                    print("="*50)
                    
                    try:
                        result_data = json.loads(result)
                        if 'error' in result_data:
                            print(f"⚠️ {result_data['error']}")
                            if 'available_routes' in result_data:
                                print("\nAvailable routes:")
                                for route in result_data['available_routes']:
                                    print(f"  • {route['from']} → {route['to']}")
                        else:
                            print(f"Found {result_data.get('total_options', 0)} flights")
                            print(f"Cheapest: {result_data.get('cheapest', {}).get('airline')} - {result_data.get('cheapest', {}).get('price')}")
                            print("\nTop flights:")
                            for flight in result_data.get('top_flights', []):
                                print(f"  • {flight['airline']}: {flight['departure']} → {flight['arrival']}")
                                print(f"    Price: {flight['price']}, Duration: {flight['duration']}")
                    except:
                        print(result[:500])
                
                elif choice == '2':
                    print("\n" + "="*50)
                    print("🏨 HOTEL SEARCH")
                    print("="*50)
                    city = input("City: ").strip()
                    
                    if not city:
                        print("❌ Please provide a city.")
                        continue
                    
                    min_stars = input("Minimum stars (1-5, optional): ").strip()
                    max_price = input("Maximum price per night (₹, optional): ").strip()
                    amenities = input("Amenities (comma-separated, optional): ").strip()
                    
                    amenities_list = [a.strip() for a in amenities.split(',')] if amenities else None
                    
                    result = hotel_tool.search(city, 
                                              min_stars=int(min_stars) if min_stars else None,
                                              max_price=float(max_price) if max_price else None,
                                              amenities=amenities_list)
                    
                    print("\n" + "="*50)
                    print("RESULTS:")
                    print("="*50)
                    
                    try:
                        result_data = json.loads(result)
                        if 'error' in result_data:
                            print(f"⚠️ {result_data['error']}")
                            if 'available_cities' in result_data:
                                print(f"\nAvailable cities: {', '.join(result_data['available_cities'])}")
                        else:
                            print(f"Found {result_data.get('total_hotels', 0)} hotels in {result_data.get('city')}")
                            print("\nTop recommendations:")
                            for hotel in result_data.get('recommendations', []):
                                print(f"  • {hotel['name']} ({hotel['star_rating']})")
                                print(f"    Price: {hotel['price_per_night']}/night")
                                print(f"    Amenities: {hotel['amenities']}")
                    except:
                        print(result[:500])
                
                elif choice == '3':
                    print("\n" + "="*50)
                    print("🏖️ TOURIST ATTRACTIONS")
                    print("="*50)
                    city = input("City: ").strip()
                    
                    if not city:
                        print("❌ Please provide a city.")
                        continue
                    
                    place_type = input("Type (optional): ").strip()
                    min_rating = input("Minimum rating (0-5, optional): ").strip()
                    
                    result = places_tool.search(city,
                                               place_type=place_type if place_type else None,
                                               min_rating=float(min_rating) if min_rating else None)
                    
                    print("\n" + "="*50)
                    print("RESULTS:")
                    print("="*50)
                    
                    try:
                        result_data = json.loads(result)
                        if 'error' in result_data:
                            print(f"⚠️ {result_data['error']}")
                            if 'available_cities' in result_data:
                                print(f"\nAvailable cities: {', '.join(result_data['available_cities'])}")
                        else:
                            print(f"Found {result_data.get('total_places', 0)} attractions in {result_data.get('city')}")
                            print("\nTop rated attractions:")
                            for place in result_data.get('top_rated', []):
                                print(f"  • {place['name']} ({place['type']})")
                                print(f"    Rating: {place['stars']} ({place['rating']}/5)")
                    except:
                        print(result[:500])
                
                elif choice == '4':
                    print("\n" + "="*50)
                    print("🌤️ WEATHER FORECAST")
                    print("="*50)
                    city = input("City: ").strip()
                    
                    if not city:
                        print("❌ Please provide a city.")
                        continue
                    
                    date = input("Date (YYYY-MM-DD, optional): ").strip()
                    
                    result = weather_tool.get_weather(city, date=date if date else None)
                    
                    print("\n" + "="*50)
                    print("RESULTS:")
                    print("="*50)
                    
                    try:
                        result_data = json.loads(result)
                        if 'error' in result_data:
                            print(f"⚠️ {result_data['error']}")
                            if 'available_cities' in result_data:
                                print(f"\nAvailable cities: {', '.join(result_data['available_cities'])}")
                        else:
                            print(f"Weather forecast for {result_data.get('city')} on {result_data.get('date')}:")
                            print(f"  • Condition: {result_data.get('weather')}")
                            print(f"  • Temperature: {result_data.get('temperature')}")
                            print(f"  • Precipitation: {result_data.get('precipitation')}")
                            print(f"  • Recommendation: {result_data.get('recommendation')}")
                    except:
                        print(result[:500])
                
                elif choice == '5':
                    print("\n" + "="*50)
                    print("💰 BUDGET CALCULATOR")
                    print("="*50)
                    
                    try:
                        flight_price = float(input("Flight price (₹): ").strip() or 5000)
                        hotel_price = float(input("Hotel price per night (₹): ").strip() or 3000)
                        num_nights = int(input("Number of nights: ").strip() or 3)
                        food_per_day = float(input("Food per day (₹): ").strip() or 1000)
                        transport_per_day = float(input("Transport per day (₹): ").strip() or 500)
                        activities_per_day = float(input("Activities per day (₹): ").strip() or 1000)
                        
                        result = budget_tool.calculate(
                            flight_price=flight_price,
                            hotel_price_per_night=hotel_price,
                            num_nights=num_nights,
                            food_per_day=food_per_day,
                            transport_per_day=transport_per_day,
                            activities_per_day=activities_per_day
                        )
                        
                        print("\n" + "="*50)
                        print("BUDGET BREAKDOWN:")
                        print("="*50)
                        
                        try:
                            result_data = json.loads(result)
                            if 'error' in result_data:
                                print(f"⚠️ {result_data['error']}")
                            else:
                                print(f"Total estimated cost: {result_data.get('total')}")
                                print(f"Average per day: {result_data.get('per_day_average')}")
                                print("\nBreakdown:")
                                for category, amount in result_data.get('budget_breakdown', {}).items():
                                    print(f"  • {category}: {amount}")
                                print("\n💡 Money-saving tips:")
                                for tip in result_data.get('money_saving_tips', []):
                                    print(f"  • {tip}")
                        except:
                            print(result[:500])
                    except ValueError:
                        print("❌ Invalid input. Please enter numbers only.")
                
                elif choice == '6':
                    print("\n" + "="*50)
                    print("🚀 CREATE TRAVEL PLAN")
                    print("="*50)
                    
                    # Get travel details
                    from_city = input("Departure city: ").strip()
                    to_city = input("Destination city: ").strip()
                    
                    if not from_city or not to_city:
                        print("❌ Please provide both cities.")
                        continue
                    
                    try:
                        num_nights = int(input("Number of nights (default 3): ").strip() or 3)
                    except:
                        num_nights = 3
                    
                    print("\n" + "="*60)
                    print(f"✈️ TRAVEL PLAN: {from_city.upper()} → {to_city.upper()}")
                    print("="*60)
                    
                    # 1. Get flight options
                    print("\n✈️ FLIGHT OPTIONS:")
                    flight_result = flight_tool.search(from_city, to_city)
                    
                    flight_price = 5000  # default
                    try:
                        flight_data = json.loads(flight_result)
                        if 'error' in flight_data:
                            print(f"  ⚠️ {flight_data['error']}")
                            if 'available_routes' in flight_data:
                                print("  Available routes:")
                                for route in flight_data['available_routes'][:3]:
                                    print(f"    • {route['from']} → {route['to']}")
                        else:
                            print(f"  Found {flight_data.get('total_options', 0)} flights")
                            if flight_data.get('top_flights'):
                                for i, flight in enumerate(flight_data['top_flights'][:3], 1):
                                    print(f"  {i}. {flight['airline']}")
                                    print(f"     Departure: {flight['departure']}")
                                    print(f"     Arrival: {flight['arrival']}")
                                    print(f"     Price: {flight['price']}")
                                    print(f"     Duration: {flight['duration']}")
                            
                            # Get flight price for budget calculation
                            if flight_data.get('cheapest') and 'price' in flight_data['cheapest']:
                                price_str = flight_data['cheapest']['price'].replace('₹', '').replace(',', '')
                                flight_price = float(price_str)
                    except:
                        print(f"  {flight_result[:200]}...")
                    
                    # 2. Get hotel options
                    print(f"\n🏨 HOTEL OPTIONS IN {to_city.upper()}:")
                    hotel_result = hotel_tool.search(to_city, min_stars=3)
                    
                    hotel_price = 3000  # default
                    try:
                        hotel_data = json.loads(hotel_result)
                        if 'error' in hotel_data:
                            print(f"  ⚠️ {hotel_data['error']}")
                            if 'available_cities' in hotel_data:
                                print(f"  Available cities: {', '.join(hotel_data['available_cities'][:5])}")
                        else:
                            print(f"  Found {hotel_data.get('total_hotels', 0)} hotels")
                            if hotel_data.get('recommendations'):
                                for i, hotel in enumerate(hotel_data['recommendations'][:3], 1):
                                    print(f"  {i}. {hotel['name']} ({hotel['star_rating']})")
                                    print(f"     Price: {hotel['price_per_night']}/night")
                                    print(f"     Amenities: {hotel['amenities']}")
                            
                            # Get hotel price for budget calculation
                            if hotel_data.get('recommendations') and hotel_data['recommendations']:
                                price_str = hotel_data['recommendations'][0]['price_per_night'].replace('₹', '').replace(',', '')
                                hotel_price = float(price_str)
                    except:
                        print(f"  {hotel_result[:200]}...")
                    
                    # 3. Get attractions
                    print(f"\n🏖️ TOP ATTRACTIONS IN {to_city.upper()}:")
                    places_result = places_tool.search(to_city)
                    
                    try:
                        places_data = json.loads(places_result)
                        if 'error' in places_data:
                            print(f"  ⚠️ {places_data['error']}")
                            if 'available_cities' in places_data:
                                print(f"  Available cities: {', '.join(places_data['available_cities'][:5])}")
                        else:
                            print(f"  Found {places_data.get('total_places', 0)} attractions")
                            if places_data.get('top_rated'):
                                for i, place in enumerate(places_data['top_rated'][:5], 1):
                                    print(f"  {i}. {place['name']} ({place['type'].title()})")
                                    print(f"     Rating: {place['stars']} ({place['rating']}/5)")
                    except:
                        print(f"  {places_result[:200]}...")
                    
                    # 4. Get weather
                    print(f"\n🌤️ WEATHER IN {to_city.upper()}:")
                    weather_result = weather_tool.get_weather(to_city)
                    
                    try:
                        weather_data = json.loads(weather_result)
                        if 'error' in weather_data:
                            print(f"  ⚠️ {weather_data['error']}")
                            if 'available_cities' in weather_data:
                                print(f"  Available cities: {', '.join(weather_data['available_cities'])}")
                        else:
                            print(f"  • Condition: {weather_data.get('weather')}")
                            print(f"  • Temperature: {weather_data.get('temperature')}")
                            print(f"  • Recommendation: {weather_data.get('recommendation')}")
                    except:
                        print(f"  {weather_result[:200]}...")
                    
                    # 5. Calculate budget
                    print(f"\n💰 ESTIMATED BUDGET FOR {num_nights} DAYS:")
                    budget_result = budget_tool.calculate(
                        flight_price=flight_price,
                        hotel_price_per_night=hotel_price,
                        num_nights=num_nights,
                        food_per_day=1000,
                        transport_per_day=500,
                        activities_per_day=1000
                    )
                    
                    try:
                        budget_data = json.loads(budget_result)
                        if 'error' in budget_data:
                            print(f"  ⚠️ {budget_data['error']}")
                        else:
                            print(f"  • Total: {budget_data.get('total')}")
                            print(f"  • Per day: {budget_data.get('per_day_average')}")
                            print("\n  💡 Money-saving tips:")
                            for tip in budget_data.get('money_saving_tips', [])[:3]:
                                print(f"    • {tip}")
                    except:
                        print(f"  {budget_result[:200]}...")
                    
                    # 6. Sample itinerary
                    print(f"\n📅 SAMPLE {num_nights}-DAY ITINERARY:")
                    days = ["Day 1: Arrival & Exploration", "Day 2: Sightseeing", "Day 3: Local Experience", 
                           "Day 4: Adventure Day", "Day 5: Relaxation & Departure"]
                    
                    for i in range(min(num_nights, 5)):
                        print(f"  {days[i]}")
                        if i == 0:
                            print("    • Check into hotel")
                            print("    • Visit nearby attractions")
                            print("    • Try local cuisine for dinner")
                        elif i == num_nights - 1:
                            print("    • Last minute shopping")
                            print("    • Check out from hotel")
                            print("    • Depart for airport")
                        else:
                            print("    • Breakfast at hotel")
                            print("    • Visit major attractions")
                            print("    • Explore local markets")
                    
                    print("\n" + "="*60)
                    print("✅ TRAVEL PLAN COMPLETE! HAVE A GREAT TRIP! ✈️")
                    print("="*60)
                
                elif choice == '7':
                    print("\n" + "="*50)
                    print("🏙️ AVAILABLE CITIES")
                    print("="*50)
                    
                    # Show available cities from data
                    print("\nFlights available from:")
                    flight_from = sorted(data_loader.flights_df['from'].unique().tolist())
                    print(f"  {', '.join(flight_from)}")
                    
                    print("\nFlights available to:")
                    flight_to = sorted(data_loader.flights_df['to'].unique().tolist())
                    print(f"  {', '.join(flight_to)}")
                    
                    print("\nHotels available in:")
                    hotel_cities = sorted(data_loader.hotels_df['city'].unique().tolist())
                    print(f"  {', '.join(hotel_cities)}")
                    
                    print("\nAttractions available in:")
                    place_cities = sorted(data_loader.places_df['city'].unique().tolist())
                    print(f"  {', '.join(place_cities)}")
                    
                    print("\nWeather available for:")
                    print("  Delhi, Mumbai, Goa, Bangalore, Chennai, Hyderabad, Kolkata, Jaipur")
                
                else:
                    print("❌ Invalid choice. Please select 1-8.")
                    
            except Exception as e:
                print(f"\n❌ Error: {str(e)}")
                print("Please try again.")
                
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        print("Please check your data files and try again.")

# =================== MAIN EXECUTION ===================

if __name__ == "__main__":
    print("Starting Travel Planning Assistant...")
    create_simple_interface()